# Filesystem Stuff

## Paths

Julia provides a lot of built-ins for working with paths in a
cross-platformy way.

In [1]:
currdir = pwd()
@show basename(currdir)
@show dirname(currdir)
readme = joinpath(currdir, "README.rst")

basename(currdir) = "administrative-scripting-with-julia"
dirname(currdir) = "/home/ninjaaron/doc"


"/home/ninjaaron/doc/administrative-scripting-with-julia/README.rst"

`joinpath` can join an arbitrary number of path elements. For splitting directory componets, there are two options. There is a `splitdir` function, which returns a 2-tuple of the basename and the parent directory:

In [2]:
splitdir(currdir)

("/home/ninjaaron/doc", "administrative-scripting-with-julia")

And in Julia 1.1+, there is a `splitpath` function, which breaks up all the path components:

In [3]:
splitpath(currdir)

5-element Array{String,1}:
 "/"                                  
 "home"                               
 "ninjaaron"                          
 "doc"                                
 "administrative-scripting-with-julia"

Here just a few other functions that work on paths:

In [4]:
@show splitext("README.rst")
@show isdir(readme)
@show isfile(readme)
st = stat(readme)

splitext("README.rst") = ("README", ".rst")
isdir(readme) = false
isfile(readme) = true


StatStruct(mode=0o100644, size=42607)

`StatStruct` instances have a lot more attributes than this. They have [all these
attributes](https://docs.julialang.org/en/v1/base/file/#Base.stat) as
well. A couple of these attributes, like `mtime` and `ctime` are in
Unix time, so it might be good mention that you can convert them to a
human readable representation with the Dates module, which is in the
standard library. It will be covered more in a later section. (Note
that this pretty-printed date is just the way it prints. It is a data
structure, not a string.)

In [5]:
import Dates
Dates.unix2datetime(st.mtime)

2019-06-18T08:09:37.918

There are many other methods available in Base which have names you should already recognize, which I won't demonstrate now. Names include: `cd`, `rm`, `mkdir`, `mkpath` (like `mkdir -p` in the shell), `symlink`, `chown`, `chmod` (careful to make sure youre mode argument is in octal, `0o644` or whatever), `cp`, `mv`, `touch`, as well as a lot of tests like `isfile`, `isdir`, `islink`, `isfifo`, etc. If you've done shell scripting, you know what they do, and you can [read the docs] if you need more. The one thing function that's missing the familiar shell name is `ls`. That's called `readdir`, after the libc function.

In [6]:
readdir()

11-element Array{String,1}:
 ".git"              
 ".gitignore"        
 ".ipynb_checkpoints"
 "1-files.ipynb"     
 "2-CLI.ipynb"       
 "3-filesystem.ipynb"
 "4-processes.ipynb" 
 "5-regex.ipynb"     
 "README.rst"        
 "base.rst"          
 "build.sh"          

There's also a
[`walkdir`](https://docs.julialang.org/en/v1/base/file/#Base.Filesystem.walkdir)
which recursively walks the directory and returns tuples of
`(rootpath, dirs, files)` which is rather handy.

There are a few things Julia still lacks in the filesystem
department. It doesn't support any kind of file globbing, but that's
easy enough to handle with regex or plain substring matching.

In [7]:
[path for path in readdir() if occursin("ipynb", path)]

6-element Array{String,1}:
 ".ipynb_checkpoints"
 "1-files.ipynb"     
 "2-CLI.ipynb"       
 "3-filesystem.ipynb"
 "4-processes.ipynb" 
 "5-regex.ipynb"     

In [8]:
# or
filter(p -> !startswith(p, "."), readdir())

8-element Array{String,1}:
 "1-files.ipynb"     
 "2-CLI.ipynb"       
 "3-filesystem.ipynb"
 "4-processes.ipynb" 
 "5-regex.ipynb"     
 "README.rst"        
 "base.rst"          
 "build.sh"          

It also weirdly lacks a function for making hard links. Bah. I guess
that's what the [C interface](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/)
is for. (I'm both thumping my chest and groaning inside as I say that,
but at least it is crazy easy to call C from Julia and is as efficient
as native calls)

In [9]:
function hardlink(oldpath, newpath)
    # calling:  int link(char *oldpath, char *newpath)
    ret_code = ccall(:link, Int32, (Cstring, Cstring), oldpath, newpath)
    # handle system errors from libc:
    systemerror("linking $oldpath -> $newpath", ret_code != 0)
end

hardlink("README.rst", "foo.txt")
st = stat("foo.txt")
@show st.nlink
rm("foo.txt")

st.nlink = 2


Course, using `ccall` sort of depends on, you know, knowing enough C
to read and understand C function declarations for simple things, and
it involves pointers and memory allocation crap if you want to do
something more serious. It's C. What did you expect?

Julia also lacks Python's easy, built-in support for compression and
archive formats, though third-party packages do exist for GZip and Zip
archives. Maybe I should work on an archiving library. Hm. Maybe not.

Anyhow, there's more than one way to skin that cat. One distinctive
feature of Julia is that is very clear after you use it a little, but
it's hard to point to any one thing, is that it wants to make it easy
to bootstrap whatever functionality you need into the language. The
`ccall` API is part of that. It is used liberally in the
implementation of OS interfaces in the standard library, as well as some of the mathematical
libraries (`ccall` works on any dynamic libraries, not just those written in C). Though they aren't shipped with Julia, the community also maintains PyCall.jl and RCall.jl, which
allow "native" calls into those runtimes for wrapping their
libraries. Macros are different example of the same thing. Language
missing a feature? Alter the semantics with a macro. Yet another
example of this "bootstrap-ability" of Julia is the ease with which it
allows the programmer to orchestrate the use of external processes.

To take the example of the above `hardlink` function, If programming
in C ain't your bag, Julia has really great support for running
external processes, so it is also possible (but rather slower) to
simply do:

In [10]:
hardlink(oldpath, newpath) = run(`link $oldpath $newpath`)

hardlink (generic function with 1 method)

So, let's transition to the section on [commands and processes](4-processes.ipynb).